## 幾種思路

思路1：TF-IDF + 機器學習分類器
直接使用TF-IDF對文本提取特徵，並使用分類器進行分類。在分類器的選擇上，可以使用SVM、LR、或者XGBoost。

思路2：FastText
FastText是入門款的詞向量，利用Facebook提供的FastText工具，可以快速構建出分類器。

思路3：WordVec + 深度學習分類器
WordVec是進階款的詞向量，並通過構建深度學習分類完成分類。深度學習分類的網絡結構可以選擇TextCNN、TextRNN或者BiLSTM。

思路4：Bert詞向量
Bert是高配款的詞向量，具有強大的建模學習能力。

## 獲取6種模型的資料集

In [1]:
import matplotlib.pyplot as plt
import os
import pandas as pd
import numpy as np
import re
path = '../data/0414/review_data(seg+pos+stopwords)_n+v+f+p.csv'
df = pd.read_csv(path)

### 檢查重複值、空值

In [2]:
#印出重複資料
print(df[df.duplicated()])

Empty DataFrame
Columns: [reviews, value, comfort, location, cleanliness, service, facilities, ws_pos_reviews, filtered, filtered_word]
Index: []


In [3]:
#移除重複值
#df = df.drop_duplicates()
#print(df.shape)

In [4]:
#印出空值資料
#df_train[df_train.isnull().T.any()]

### 切分為6個資料集

In [5]:
def split_df(df):
    df_value = df[['value','filtered_word']]
    df_value.rename(columns={'value': 'label'}, inplace=True)
    df_comfort = df[['comfort','filtered_word']]
    df_comfort.rename(columns={'comfort': 'label'}, inplace=True)
    df_location = df[['location','filtered_word']]
    df_location.rename(columns={'location': 'label'}, inplace=True)
    df_cleanliness = df[['cleanliness','filtered_word']]
    df_cleanliness.rename(columns={'cleanliness': 'label'}, inplace=True)
    df_service = df[['service','filtered_word']]
    df_service.rename(columns={'service': 'label'}, inplace=True)
    df_facilities = df[['facilities','filtered_word']]
    df_facilities.rename(columns={'facilities': 'label'}, inplace=True)
    return df_value, df_comfort, df_location, df_cleanliness, df_service, df_facilities

In [6]:
#df_value_train, df_comfort_train, df_location_train, df_cleanliness_train, df_service_train ,df_facilities_train = split_df(df_train)
#df_value_test, df_comfort_test, df_location_test, df_cleanliness_test, df_service_test, df_facilities_test = split_df(df_test)
df_value, df_comfort, df_location, df_cleanliness, df_service ,df_facilities = split_df(df)

C:\Users\g3190\AppData\Local\Temp/ipykernel_19868/3099386144.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_value.rename(columns={'value': 'label'}, inplace=True)
C:\Users\g3190\AppData\Local\Temp/ipykernel_19868/3099386144.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_comfort.rename(columns={'comfort': 'label'}, inplace=True)
C:\Users\g3190\AppData\Local\Temp/ipykernel_19868/3099386144.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-v

In [7]:
df_location

,label,filtered_word
0,0.0,"價格(Na),合理(VH),舒適(VH),房間(Nc),老闆娘(Na),人(Na),好(VH..."
1,0.0,"內部(Ncd),房間(Nc),乾淨(VH),場地(Na),團體(Na),使用(VC),覺得(..."
2,0.0,"房間(Nc),小(VH),美中不足(VH),乾(VH),濕(VH),分離(VHC)"
3,0.0,"房子(Na),設計(VC),棒(VH),房間(Nc),採光(Na),好(VH),大廳(Nc)..."
4,0.0,"Cp值(FW),高(VH),乾淨(VH),舒適(VH),空間(Na),大樓(Na),下(Nc..."
...,...,...
1267,1.0,"港式(Na),飲茶(VA),餐廳(Nc),口味(Na),棒(VH),環境(Na),乾淨(VH..."
1268,0.0,"場地(Na),氣派(Na),丁香魚(Na),酥脆(VH),服務(VC),親切(VH),蠟味(..."
1269,1.0,"交通(Na),方便(VH),地下室(Nc),停車場(Nc),良好(VH),菜色(Na),好(..."
1270,1.0,"地點(Na),佳(VH),離(P),逢甲(Nb),夜市(Nc),老闆娘(Na),親切(VH)..."


### 清理資料(移除詞性標註的文字)

In [8]:
def remove_N_comma(sentence):
    # 把後面(N..)(V..)(F..)拿掉
    sentence = str(sentence)
    pattern = re.compile(r"\([N,V,F,P].*?\)") #移除詞性標示
    sentence = re.sub(pattern, '', sentence)
    pattern = re.compile(r",") #將逗號替換為空格
    sentence = re.sub(pattern, ' ', sentence)
    return sentence
pd.options.mode.chained_assignment = None  # 忽略警告

In [9]:
#訓練集
df_location['filtered_word'] = df_location.apply(lambda x: remove_N_comma(x['filtered_word']),axis=1)

In [10]:
print(df_location.shape)
df_location

(1272, 2)


,label,filtered_word
0,0.0,價格 合理 舒適 房間 老闆娘 人 好 做 早餐 旅客 重點 早餐 吃到飽
1,0.0,內部 房間 乾淨 場地 團體 使用 覺得 棒
2,0.0,房間 小 美中不足 乾 濕 分離
3,0.0,房子 設計 棒 房間 採光 好 大廳 挑高 氣派 房價 合理 台東 住 民宿
4,0.0,Cp值 高 乾淨 舒適 空間 大樓 下 免費 吐司 咖啡 老闆 回復 速度
...,...,...
1267,1.0,港式 飲茶 餐廳 口味 棒 環境 乾淨 機車 汽車 停車位 位於 高鐵 附近 適合 宴客
1268,0.0,場地 氣派 丁香魚 酥脆 服務 親切 蠟味 蘿蔔糕 份量 一些 好 牛肉粥 好吃
1269,1.0,交通 方便 地下室 停車場 良好 菜色 好 空間 設計好 說 一流 飯店
1270,1.0,地點 佳 離 逢甲 夜市 老闆娘 親切 服務 房間 舒適 浴室 乾淨


## 模型架構

### 套件引用

In [11]:
#import package
#轉向量用
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer 
from scipy.sparse import coo_matrix

from imblearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import cross_validate
import pickle #儲存模型用
from sklearn.model_selection import train_test_split
#類別採樣
import imblearn.over_sampling as over_sampling
import imblearn.under_sampling as under_sampling
import imblearn.combine as combine
from imblearn.pipeline import make_pipeline as make_pipeline_imb


#模型
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.ensemble import AdaBoostClassifier
#from xgboost.sklearn import XGBClassifier

#模型效能表現
import sklearn.metrics as metrics

### 顯示訓練結果

In [12]:
def classification_report(y_test, pre):
    #混淆矩陣
    confusion = metrics.confusion_matrix(y_test, pre)
    TP = confusion[1,1]
    TN = confusion[0,0]
    FP = confusion[0,1]
    FN = confusion[1,0]
    print("TP:",TP)
    print("TN:",TN)
    print("FP:",FP)
    print("FN:",FN)
    #Accuracy
    accuracy = (TP+TN)/float(TP+TN+FN+FP)
    print("Accuracy：", accuracy)
    #Sensitivity(Recall)
    recall = TP/float(TP+FN)
    print("Recall：", recall)
    #Specificity
    specificity = TN/float(TN+FP)
    print("Specificity：", specificity)
    #Precision
    precision = TP/float(TP+FP)
    print("Precision：", precision)
    #f1-score
    f1_score = ((2*precision*recall)/(precision+recall))
    print("f1_score：", f1_score)
    #AUC
    print("AUC：", metrics.roc_auc_score(y_test, pre))

In [13]:
df_location

,label,filtered_word
0,0.0,價格 合理 舒適 房間 老闆娘 人 好 做 早餐 旅客 重點 早餐 吃到飽
1,0.0,內部 房間 乾淨 場地 團體 使用 覺得 棒
2,0.0,房間 小 美中不足 乾 濕 分離
3,0.0,房子 設計 棒 房間 採光 好 大廳 挑高 氣派 房價 合理 台東 住 民宿
4,0.0,Cp值 高 乾淨 舒適 空間 大樓 下 免費 吐司 咖啡 老闆 回復 速度
...,...,...
1267,1.0,港式 飲茶 餐廳 口味 棒 環境 乾淨 機車 汽車 停車位 位於 高鐵 附近 適合 宴客
1268,0.0,場地 氣派 丁香魚 酥脆 服務 親切 蠟味 蘿蔔糕 份量 一些 好 牛肉粥 好吃
1269,1.0,交通 方便 地下室 停車場 良好 菜色 好 空間 設計好 說 一流 飯店
1270,1.0,地點 佳 離 逢甲 夜市 老闆娘 親切 服務 房間 舒適 浴室 乾淨


### 切分訓練、測試數據

In [14]:
from sklearn.model_selection import train_test_split
def split_label(df,seed):
    X = df.filtered_word.tolist()
    y = df.label
    X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=1/3,random_state=seed)
    return X_train,X_test,y_train,y_test

In [15]:
X_train_1,X_test_1,y_train_1,y_test_1 = split_label(df_location,1)
X_train_2,X_test_2,y_train_2,y_test_2 = split_label(df_location,2)
X_train_3,X_test_3,y_train_3,y_test_3 = split_label(df_location,3)

In [16]:
s1 = pd.Series(y_train_1)
freq1 = s1.value_counts() 
print(freq1) 
s2 = pd.Series(y_train_2)
freq2 = s2.value_counts() 
print(freq2) 
s3 = pd.Series(y_train_3)
freq3 = s3.value_counts() 
print(freq3) 

0.0    608
1.0    240
Name: label, dtype: int64
0.0    591
1.0    257
Name: label, dtype: int64
0.0    608
1.0    240
Name: label, dtype: int64


### 模型設計

#### (1) baseline

In [17]:
def SVM_model(X_train,X_test,y_train,y_test):
    print("SVM baseline")
    #切分數據集
    #X_train,y_train,X_test,y_test = split_label(df_train, df_test)
    #模型架構
    model = make_pipeline_imb(TfidfVectorizer(max_df=0.8,min_df=5,dtype=np.float32), svm.SVC(kernel='linear'))
    model.fit(X_train, y_train)
    #模型預測
    pre = model.predict(X_test)
    #麼行評估
    classification_report(y_test, pre)
    print("\n")

In [18]:
def LR_model(X_train,X_test,y_train,y_test):
    print("LR baseline")
    #切分數據集
    #X_train,y_train,X_test,y_test = split_label(df_train, df_test)
    #模型架構
    model = make_pipeline_imb(TfidfVectorizer(max_df=0.8,min_df=5,dtype=np.float32), LogisticRegression(random_state=0))
    model.fit(X_train, y_train)
    #模型預測
    pre = model.predict(X_test)
    #麼行評估
    classification_report(y_test, pre)
    print("\n")

In [19]:
def RF_model(X_train,X_test,y_train,y_test):
    print("RF baseline")
    #切分數據集
    #X_train,y_train,X_test,y_test = split_label(df_train, df_test)
    #模型架構
    model = make_pipeline_imb(TfidfVectorizer(max_df=0.8,min_df=5,dtype=np.float32), RandomForestClassifier(max_depth=2, random_state=0))
    model.fit(X_train, y_train)
    #模型預測
    pre = model.predict(X_test)
    #麼行評估
    classification_report(y_test, pre)
    print("\n")

In [20]:
def AdaBoost_model(X_train,X_test,y_train,y_test):
    print("AdaBoost_model")
    #切分數據集
    #X_train,y_train,X_test,y_test = split_label(df_train, df_test)
    #模型架構
    model = make_pipeline_imb(TfidfVectorizer(), AdaBoostClassifier(n_estimators=200))
    model.fit(X_train, y_train)
    #模型預測
    pre = model.predict(X_test)
    #麼行評估
    classification_report(y_test, pre)
    print("\n")

#### (2)執行採樣 => 解決類別不平衡 (SVM)

In [21]:
def SVM_model2(X_train,X_test,y_train,y_test):
    print("ADASYN")
    #切分數據集
    #X_train,y_train,X_test,y_test = split_label(df_train, df_test)
    #模型架構
    model = make_pipeline_imb(TfidfVectorizer(), over_sampling.ADASYN(), svm.SVC(kernel='linear'))
    model.fit(X_train, y_train)
    #模型預測
    pre = model.predict(X_test)
    #麼行評估
    classification_report(y_test, pre)
    print("\n")

In [22]:
def SVM_model3(X_train,X_test,y_train,y_test):
    print("SMOTE")
    #切分數據集
    #X_train,y_train,X_test,y_test = split_label(df_train, df_test)
    #模型架構
    model = make_pipeline_imb(TfidfVectorizer(), over_sampling.SMOTE(), svm.SVC(kernel='linear'))
    model.fit(X_train, y_train)
    #模型預測
    pre = model.predict(X_test)
    #麼行評估
    classification_report(y_test, pre)
    print("\n")

In [23]:
def SVM_model4(X_train,X_test,y_train,y_test):
    print("RandomOverSampler")
    #切分數據集
    #X_train,y_train,X_test,y_test = split_label(df_train, df_test)
    #模型架構
    model = make_pipeline_imb(TfidfVectorizer(), over_sampling.RandomOverSampler(), svm.SVC(kernel='linear'))
    model.fit(X_train, y_train)
    #模型預測
    pre = model.predict(X_test)
    #麼行評估
    classification_report(y_test, pre)
    print("\n")

In [24]:
def SVM_model5(X_train,X_test,y_train,y_test):
    print("RandomUnderSampler")
    #切分數據集
    #X_train,y_train,X_test,y_test = split_label(df_train, df_test)
    #模型架構
    model = make_pipeline_imb(TfidfVectorizer(), under_sampling.RandomUnderSampler(), svm.SVC(kernel='linear'))
    model.fit(X_train, y_train)
    #模型預測
    pre = model.predict(X_test)
    #麼行評估
    classification_report(y_test, pre)
    print("\n")

#### (3)執行採樣 => 解決類別不平衡 (Adaboost)

In [25]:
def AdaBoost_model2(X_train,X_test,y_train,y_test):
    print("ADASYN")
    #切分數據集
    #X_train,y_train,X_test,y_test = split_label(df_train, df_test)
    #模型架構
    model = make_pipeline_imb(TfidfVectorizer(), over_sampling.ADASYN(), AdaBoostClassifier(n_estimators=200))
    model.fit(X_train, y_train)
    #模型預測
    pre = model.predict(X_test)
    #麼行評估
    classification_report(y_test, pre)
    print("\n")

In [26]:
def AdaBoost_model3(X_train,X_test,y_train,y_test):
    print("SMOTE")
    #切分數據集
    #X_train,y_train,X_test,y_test = split_label(df_train, df_test)
    #模型架構
    model = make_pipeline_imb(TfidfVectorizer(), over_sampling.SMOTE(), AdaBoostClassifier(n_estimators=200))
    model.fit(X_train, y_train)
    #模型預測
    pre = model.predict(X_test)
    #麼行評估
    classification_report(y_test, pre)
    print("\n")

In [27]:
def AdaBoost_model4(X_train,X_test,y_train,y_test):
    print("RandomOverSampler")
    #切分數據集
    #X_train,y_train,X_test,y_test = split_label(df_train, df_test)
    #模型架構
    model = make_pipeline_imb(TfidfVectorizer(), over_sampling.RandomOverSampler(), AdaBoostClassifier(n_estimators=200))
    model.fit(X_train, y_train)
    #模型預測
    pre = model.predict(X_test)
    #麼行評估
    classification_report(y_test, pre)
    print("\n")

In [28]:
def AdaBoost_model5(X_train,X_test,y_train,y_test):
    print("RandomUnderSampler")
    #切分數據集
    #X_train,y_train,X_test,y_test = split_label(df_train, df_test)
    #模型架構
    model = make_pipeline_imb(TfidfVectorizer(), under_sampling.RandomUnderSampler(), AdaBoostClassifier(n_estimators=200))
    model.fit(X_train, y_train)
    #模型預測
    pre = model.predict(X_test)
    #麼行評估
    classification_report(y_test, pre)
    print("\n")

## 模型訓練&結果

### 資料集1

#### baseline

In [29]:
SVM_model(X_train_1,X_test_1,y_train_1,y_test_1)
LR_model(X_train_1,X_test_1,y_train_1,y_test_1)
RF_model(X_train_1,X_test_1,y_train_1,y_test_1)
AdaBoost_model(X_train_1,X_test_1,y_train_1,y_test_1)

SVM baseline
TP: 93
TN: 281
FP: 7
FN: 43
Accuracy： 0.8820754716981132
Recall： 0.6838235294117647
Specificity： 0.9756944444444444
Precision： 0.93
f1_score： 0.788135593220339
AUC： 0.8297589869281046


LR baseline
TP: 69
TN: 281
FP: 7
FN: 67
Accuracy： 0.8254716981132075
Recall： 0.5073529411764706
Specificity： 0.9756944444444444
Precision： 0.9078947368421053
f1_score： 0.6509433962264151
AUC： 0.7415236928104575


RF baseline
TP: 0
TN: 288
FP: 0
FN: 136
Accuracy： 0.6792452830188679
Recall： 0.0
Specificity： 1.0
Precision： nan
f1_score： nan
AUC： 0.5


AdaBoost_model


C:\Users\g3190\AppData\Local\Temp/ipykernel_19868/3777238089.py:22: RuntimeWarning: invalid value encountered in true_divide
  precision = TP/float(TP+FP)


TP: 97
TN: 268
FP: 20
FN: 39
Accuracy： 0.8608490566037735
Recall： 0.7132352941176471
Specificity： 0.9305555555555556
Precision： 0.8290598290598291
f1_score： 0.766798418972332
AUC： 0.8218954248366014




#### 處理類別不平衡(SVM)

In [30]:
SVM_model2(X_train_1,X_test_1,y_train_1,y_test_1)
SVM_model3(X_train_1,X_test_1,y_train_1,y_test_1)
SVM_model4(X_train_1,X_test_1,y_train_1,y_test_1)
SVM_model5(X_train_1,X_test_1,y_train_1,y_test_1)

ADASYN
TP: 104
TN: 270
FP: 18
FN: 32
Accuracy： 0.8820754716981132
Recall： 0.7647058823529411
Specificity： 0.9375
Precision： 0.8524590163934426
f1_score： 0.8062015503875968
AUC： 0.8511029411764706


SMOTE
TP: 101
TN: 271
FP: 17
FN: 35
Accuracy： 0.8773584905660378
Recall： 0.7426470588235294
Specificity： 0.9409722222222222
Precision： 0.8559322033898306
f1_score： 0.7952755905511811
AUC： 0.8418096405228759


RandomOverSampler
TP: 104
TN: 270
FP: 18
FN: 32
Accuracy： 0.8820754716981132
Recall： 0.7647058823529411
Specificity： 0.9375
Precision： 0.8524590163934426
f1_score： 0.8062015503875968
AUC： 0.8511029411764706


RandomUnderSampler
TP: 110
TN: 245
FP: 43
FN: 26
Accuracy： 0.8372641509433962
Recall： 0.8088235294117647
Specificity： 0.8506944444444444
Precision： 0.7189542483660131
f1_score： 0.7612456747404844
AUC： 0.8297589869281046




#### 處理類別不平衡(Adaboost)

In [31]:
AdaBoost_model2(X_train_1,X_test_1,y_train_1,y_test_1)
AdaBoost_model3(X_train_1,X_test_1,y_train_1,y_test_1)
AdaBoost_model4(X_train_1,X_test_1,y_train_1,y_test_1)
AdaBoost_model5(X_train_1,X_test_1,y_train_1,y_test_1)

ADASYN
TP: 102
TN: 271
FP: 17
FN: 34
Accuracy： 0.8797169811320755
Recall： 0.75
Specificity： 0.9409722222222222
Precision： 0.8571428571428571
f1_score： 0.7999999999999999
AUC： 0.845486111111111


SMOTE
TP: 105
TN: 265
FP: 23
FN: 31
Accuracy： 0.8726415094339622
Recall： 0.7720588235294118
Specificity： 0.9201388888888888
Precision： 0.8203125
f1_score： 0.7954545454545454
AUC： 0.8460988562091503


RandomOverSampler
TP: 103
TN: 263
FP: 25
FN: 33
Accuracy： 0.8632075471698113
Recall： 0.7573529411764706
Specificity： 0.9131944444444444
Precision： 0.8046875
f1_score： 0.7803030303030303
AUC： 0.8352736928104575


RandomUnderSampler
TP: 108
TN: 237
FP: 51
FN: 28
Accuracy： 0.8136792452830188
Recall： 0.7941176470588235
Specificity： 0.8229166666666666
Precision： 0.6792452830188679
f1_score： 0.7322033898305084
AUC： 0.808517156862745




### 資料集2

#### baseline

In [32]:
SVM_model(X_train_2,X_test_2,y_train_2,y_test_2)
LR_model(X_train_2,X_test_2,y_train_2,y_test_2)
RF_model(X_train_2,X_test_2,y_train_2,y_test_2)
AdaBoost_model(X_train_2,X_test_2,y_train_2,y_test_2)

SVM baseline
TP: 84
TN: 293
FP: 12
FN: 35
Accuracy： 0.8891509433962265
Recall： 0.7058823529411765
Specificity： 0.9606557377049181
Precision： 0.875
f1_score： 0.7813953488372093
AUC： 0.8332690453230474


LR baseline
TP: 67
TN: 294
FP: 11
FN: 52
Accuracy： 0.8514150943396226
Recall： 0.5630252100840336
Specificity： 0.9639344262295082
Precision： 0.8589743589743589
f1_score： 0.6802030456852792
AUC： 0.7634798181567709


RF baseline
TP: 1
TN: 305
FP: 0
FN: 118
Accuracy： 0.7216981132075472
Recall： 0.008403361344537815
Specificity： 1.0
Precision： 1.0
f1_score： 0.016666666666666666
AUC： 0.5042016806722689


AdaBoost_model
TP: 96
TN: 275
FP: 30
FN: 23
Accuracy： 0.875
Recall： 0.8067226890756303
Specificity： 0.9016393442622951
Precision： 0.7619047619047619
f1_score： 0.7836734693877551
AUC： 0.8541810166689626




#### 處理類別不平衡(SVM)

In [33]:
SVM_model2(X_train_2,X_test_2,y_train_2,y_test_2)
SVM_model3(X_train_2,X_test_2,y_train_2,y_test_2)
SVM_model4(X_train_2,X_test_2,y_train_2,y_test_2)
SVM_model5(X_train_2,X_test_2,y_train_2,y_test_2)

ADASYN
TP: 93
TN: 285
FP: 20
FN: 26
Accuracy： 0.8915094339622641
Recall： 0.7815126050420168
Specificity： 0.9344262295081968
Precision： 0.8230088495575221
f1_score： 0.8017241379310344
AUC： 0.8579694172751068


SMOTE
TP: 89
TN: 288
FP: 17
FN: 30
Accuracy： 0.8891509433962265
Recall： 0.7478991596638656
Specificity： 0.9442622950819672
Precision： 0.839622641509434
f1_score： 0.7911111111111112
AUC： 0.8460807273729164


RandomOverSampler
TP: 90
TN: 286
FP: 19
FN: 29
Accuracy： 0.8867924528301887
Recall： 0.7563025210084033
Specificity： 0.9377049180327869
Precision： 0.8256880733944955
f1_score： 0.7894736842105264
AUC： 0.847003719520595


RandomUnderSampler
TP: 94
TN: 263
FP: 42
FN: 25
Accuracy： 0.8419811320754716
Recall： 0.7899159663865546
Specificity： 0.8622950819672132
Precision： 0.6911764705882353
f1_score： 0.7372549019607842
AUC： 0.8261055241768838




#### 處理類別不平衡(Adaboost)

In [34]:
AdaBoost_model2(X_train_2,X_test_2,y_train_2,y_test_2)
AdaBoost_model3(X_train_2,X_test_2,y_train_2,y_test_2)
AdaBoost_model4(X_train_2,X_test_2,y_train_2,y_test_2)
AdaBoost_model5(X_train_2,X_test_2,y_train_2,y_test_2)

ADASYN
TP: 94
TN: 276
FP: 29
FN: 25
Accuracy： 0.8726415094339622
Recall： 0.7899159663865546
Specificity： 0.9049180327868852
Precision： 0.7642276422764228
f1_score： 0.7768595041322314
AUC： 0.8474169995867198


SMOTE
TP: 91
TN: 270
FP: 35
FN: 28
Accuracy： 0.8514150943396226
Recall： 0.7647058823529411
Specificity： 0.8852459016393442
Precision： 0.7222222222222222
f1_score： 0.7428571428571428
AUC： 0.8249758919961427


RandomOverSampler
TP: 90
TN: 277
FP: 28
FN: 29
Accuracy： 0.8655660377358491
Recall： 0.7563025210084033
Specificity： 0.9081967213114754
Precision： 0.7627118644067796
f1_score： 0.7594936708860759
AUC： 0.8322496211599394


RandomUnderSampler
TP: 92
TN: 258
FP: 47
FN: 27
Accuracy： 0.8254716981132075
Recall： 0.773109243697479
Specificity： 0.8459016393442623
Precision： 0.6618705035971223
f1_score： 0.7131782945736435
AUC： 0.8095054415208707




### 資料集3

#### baseline

In [35]:
SVM_model(X_train_3,X_test_3,y_train_3,y_test_3)
LR_model(X_train_3,X_test_3,y_train_3,y_test_3)
RF_model(X_train_3,X_test_3,y_train_3,y_test_3)
AdaBoost_model(X_train_3,X_test_3,y_train_3,y_test_3)

SVM baseline
TP: 91
TN: 283
FP: 5
FN: 45
Accuracy： 0.8820754716981132
Recall： 0.6691176470588235
Specificity： 0.9826388888888888
Precision： 0.9479166666666666
f1_score： 0.7844827586206896
AUC： 0.8258782679738561


LR baseline
TP: 72
TN: 284
FP: 4
FN: 64
Accuracy： 0.839622641509434
Recall： 0.5294117647058824
Specificity： 0.9861111111111112
Precision： 0.9473684210526315
f1_score： 0.679245283018868
AUC： 0.7577614379084967


RF baseline
TP: 0
TN: 288
FP: 0
FN: 136
Accuracy： 0.6792452830188679
Recall： 0.0
Specificity： 1.0
Precision： nan
f1_score： nan
AUC： 0.5


AdaBoost_model


C:\Users\g3190\AppData\Local\Temp/ipykernel_19868/3777238089.py:22: RuntimeWarning: invalid value encountered in true_divide
  precision = TP/float(TP+FP)


TP: 101
TN: 267
FP: 21
FN: 35
Accuracy： 0.8679245283018868
Recall： 0.7426470588235294
Specificity： 0.9270833333333334
Precision： 0.8278688524590164
f1_score： 0.7829457364341087
AUC： 0.8348651960784315




#### 處理類別不平衡(SVM)

In [36]:
SVM_model2(X_train_3,X_test_3,y_train_3,y_test_3)
SVM_model3(X_train_3,X_test_3,y_train_3,y_test_3)
SVM_model4(X_train_3,X_test_3,y_train_3,y_test_3)
SVM_model5(X_train_3,X_test_3,y_train_3,y_test_3)

ADASYN
TP: 101
TN: 276
FP: 12
FN: 35
Accuracy： 0.8891509433962265
Recall： 0.7426470588235294
Specificity： 0.9583333333333334
Precision： 0.8938053097345132
f1_score： 0.8112449799196788
AUC： 0.8504901960784315


SMOTE
TP: 99
TN: 279
FP: 9
FN: 37
Accuracy： 0.8915094339622641
Recall： 0.7279411764705882
Specificity： 0.96875
Precision： 0.9166666666666666
f1_score： 0.8114754098360656
AUC： 0.8483455882352942


RandomOverSampler
TP: 100
TN: 279
FP: 9
FN: 36
Accuracy： 0.8938679245283019
Recall： 0.7352941176470589
Specificity： 0.96875
Precision： 0.9174311926605505
f1_score： 0.8163265306122449
AUC： 0.8520220588235294


RandomUnderSampler
TP: 107
TN: 261
FP: 27
FN: 29
Accuracy： 0.8679245283018868
Recall： 0.7867647058823529
Specificity： 0.90625
Precision： 0.7985074626865671
f1_score： 0.7925925925925927
AUC： 0.8465073529411764




#### 處理類別不平衡(Adaboost)

In [37]:
AdaBoost_model2(X_train_3,X_test_3,y_train_3,y_test_3)
AdaBoost_model3(X_train_3,X_test_3,y_train_3,y_test_3)
AdaBoost_model4(X_train_3,X_test_3,y_train_3,y_test_3)
AdaBoost_model5(X_train_3,X_test_3,y_train_3,y_test_3)

ADASYN
TP: 103
TN: 269
FP: 19
FN: 33
Accuracy： 0.8773584905660378
Recall： 0.7573529411764706
Specificity： 0.9340277777777778
Precision： 0.8442622950819673
f1_score： 0.7984496124031008
AUC： 0.8456903594771242


SMOTE
TP: 101
TN: 270
FP: 18
FN: 35
Accuracy： 0.875
Recall： 0.7426470588235294
Specificity： 0.9375
Precision： 0.8487394957983193
f1_score： 0.792156862745098
AUC： 0.8400735294117647


RandomOverSampler
TP: 101
TN: 269
FP: 19
FN: 35
Accuracy： 0.8726415094339622
Recall： 0.7426470588235294
Specificity： 0.9340277777777778
Precision： 0.8416666666666667
f1_score： 0.7890625
AUC： 0.8383374183006536


RandomUnderSampler
TP: 110
TN: 248
FP: 40
FN: 26
Accuracy： 0.8443396226415094
Recall： 0.8088235294117647
Specificity： 0.8611111111111112
Precision： 0.7333333333333333
f1_score： 0.7692307692307693
AUC： 0.8349673202614379


